In [835]:
import re
from tabulate import tabulate
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup
from prettytable import PrettyTable

In [876]:
def currency_info(doc):
    cur = doc.find_all('div',{'class':'currency__CurrencySection-sc-1xymln9-3 kLTJuY'})
    cur_dict = {'Name':[],'Code':[], 'Links':[]} 
    for i in range(1,len(cur)):
        cur_in = cur[i].find_all('div',{'class':'currency__CurrencyContainer-sc-1xymln9-4 hHKobN'})
        cur_in_a = cur_in[0].find_all('a',{'class':'currency__ListLink-sc-1xymln9-6 epSNTh'})
        for j in range(len(cur_in_a)):           
                    cur_in_a_link = cur_in_a[j]['href']
                    cur_in_a_cc = (cur_in_a[j].find_all('span',{'class':'currencyCode'}))[0].text.split()[0]
                    cur_in_a_name = ' '.join(cur_in_a[j].text.split()[2:7])
                    cur_dict['Name'].append(cur_in_a_name)
                    cur_dict['Code'].append(cur_in_a_cc)
                    cur_dict['Links'].append('https://www.xe.com'+cur_in_a_link)

    df = pd.DataFrame(cur_dict)
    return df      

In [862]:
#currency_info()

In [863]:
def conversion(c1,c2,amount):
        code_url = 'https://www.xe.com/currencyconverter/convert/?Amount='+ str(amount) + '&From=' + c1 + '&To=' + c2 
        response = requests.get(code_url)
        doc = BeautifulSoup(response.text, 'html.parser')
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(code_url))
        conv = doc.find_all('p',{'class':'result__BigRate-sc-1bsijpp-1 iGrAod'})
        time = doc.find_all('div',{'class':'result__LiveSubText-sc-1bsijpp-2 iKYXwX'})
        return conv[0].text + " [" + time[0].text + " ]"
    

In [864]:
 def stats_conv(c1,c2,amount):
        code_url = 'https://www.xe.com/currencyconverter/convert/?Amount='+ str(amount) + '&From=' + c1 + '&To=' + c2 
        response = requests.get(code_url)
        doc = BeautifulSoup(response.text, 'html.parser')
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(code_url))
        stats= doc.find_all('h2',{'class':'heading__Heading1-n07sti-0 heading__Heading2-n07sti-1 iXbZUl'})    
        sub = doc.find_all('th',{'class':'stats-table__ColumnHeading-sc-1uiw32l-2 kqQaYk'})
        row = doc.find_all('div',{'class':'tooltip__TooltipContainer-sc-74vpd9-0 cKhvsP'})
        row_values = doc.find_all('td')
        stats_table = PrettyTable()
        stats_table.field_names = [" ",sub[0].text,sub[1].text]
        stats_table.title  = stats[0].text
        stats_table.add_row([row[7].text[0:4],row_values[44].text,row_values[45].text])
        stats_table.add_row([row[8].text[0:3],row_values[46].text,row_values[47].text])
        stats_table.add_row([row[9].text[0:7],row_values[48].text,row_values[49].text])
        stats_table.add_row([row[10].text[0:10],row_values[50].text,row_values[50].text])
        
        print(stats_table)
        

In [865]:
def stat1(doc):
        Heading = doc.find_all('h4',{'class','heading__Heading1-n07sti-0 heading__Heading4-n07sti-3 iXbZUl'})    
        stats_curr_table = PrettyTable()
        stats_curr_table.field_names = [" "," Details"]
        stats_curr_table.title = Heading[0].text  #1 also
        Table = doc.find_all('table',{'class','currency__InfoTable-sc-4472af-2 bilixD'})
        stats_curr_table.add_row((Table[0].find_all('tr')[0].text[0:4] + " " + Table[0].find_all('tr')[0].text[4:]).split(" ",1))
        stats_curr_table.add_row((Table[0].find_all('tr')[1].text[0:6] + " " + Table[0].find_all('tr')[1].text[6:]).split(" ",1))
        L1 = (Table[0].find_all('tr')[2].text[0:10] +" " + Table[0].find_all('tr')[2].text[10:]).split(" ",2)
        stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + L1[2:]  ) 
        L2 = (Table[0].find_all('tr')[3].text[0:17] + " " + Table[0].find_all('tr')[3].text[17:]).split()
        if(len((" ".join(L2[0:3])).split("-") + L2[3:]) == 1):
            stats_curr_table.add_row((" ".join(L2[0:3])).split("-") + L2[3:] + [' '])
        else:
            stats_curr_table.add_row((" ".join(L2[0:3])).split("-") + L2[3:])   


        return stats_curr_table

def stat2(doc):
        Heading = doc.find_all('h4',{'class','heading__Heading1-n07sti-0 heading__Heading4-n07sti-3 iXbZUl'})    
        stats_curr_table = PrettyTable()
        stats_curr_table.field_names = [" "," Details"]
        stats_curr_table.title = Heading[1].text  #1 also
        Table = doc.find_all('table',{'class','currency__InfoTable-sc-4472af-2 bilixD'})
        
                
        if(Table[1].find_all('tr')[0].text[0:5].split("-")[0] == 'Users'):
            list1 = (Table[1].find_all('tr')[0].text[0:5].split("-"))
            list2 = (doc.find_all('span',{'class':'currency__OneLineSpan-sc-4472af-3 bzUWsw'})[0].text.split("-"))
            stats_curr_table.add_row(list1+list2)
            print (stats_curr_table)
        
        elif((Table[1].find_all('tr')[0].text[0:12] +" " + Table[1].find_all('tr')[0].text[12:]).split(" ",2)[0] == 'Central bank'):
            L2 = (Table[1].find_all('tr')[0].text[0:12] +" " + Table[1].find_all('tr')[0].text[12:]).split(" ",2)
            stats_curr_table.add_row((" ".join(L2[0:2])).split("-") + L2[2:]  )
            list1 = (Table[1].find_all('tr')[1].text[0:5].split("-"))
            list2 = (doc.find_all('span',{'class':'currency__OneLineSpan-sc-4472af-3 bzUWsw'})[0].text.split("-"))
            stats_curr_table.add_row(list1+list2)
            print (stats_curr_table)

        
        
        elif((Table[1].find_all('tr')[0].text[0:5] + " " + Table[1].find_all('tr')[0].text[5:]).split(" ")[0] == 'Bank'):
            L1 = (Table[1].find_all('tr')[0].text[0:10] + " " + Table[1].find_all('tr')[0].text[10:]).split(" ",2)
            if(re.search('Rarely',L1[1:][0])):
                x = L1[1:][0].find('Rarely')
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + (L1[2:][0][:x] + " " + L1[2:][0][x:]).split("-"))
            else:
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + L1[2:]  ) 
            L2 = (Table[1].find_all('tr')[1].text[0:12] +" " + Table[1].find_all('tr')[1].text[12:]).split(" ",2)
            stats_curr_table.add_row((" ".join(L2[0:2])).split("-") + L2[2:]  )
            list1 = (Table[1].find_all('tr')[2].text[0:5].split("-"))
            list2 = (doc.find_all('span',{'class':'currency__OneLineSpan-sc-4472af-3 bzUWsw'})[0].text.split("-"))
            stats_curr_table.add_row(list1+list2)
          
        
        elif ((Table[1].find_all('tr')[0].text[0:5] + " " + Table[1].find_all('tr')[0].text[5:]).split(" ",1)[0] == 'coins'):
            L0 = (Table[1].find_all('tr')[0].text[0:5] + " " + Table[1].find_all('tr')[0].text[5:]).split(" ",1)
            if(re.search('Rarely',L0[1:][0])):
                x = L0[1:][0].find('Rarely')
                stats_curr_table.add_row(" ".join(L0[0:1]).split("-") + (L0[1:][0][:x] + " " + L0[1:][0][x:]).split('-'))
            else:
                stats_curr_table.add_row((Table[1].find_all('tr')[0].text[0:5] + " " + Table[1].find_all('tr')[0].text[5:]).split(" ",1))

            L1 = (Table[1].find_all('tr')[1].text[0:10] + " " + Table[1].find_all('tr')[1].text[10:]).split(" ",2)

            if(re.search('Rarely',L1[2:][0])):
                x = L1[2:][0].find('Rarely')
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + (L1[2:][0][:x] + " " + L1[2:][0][x:]).split("-"))
            else:
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + L1[2:]  ) 
            
            
            L2 = (Table[1].find_all('tr')[2].text[0:12] +" " + Table[1].find_all('tr')[2].text[12:]).split(" ",2)
            stats_curr_table.add_row((" ".join(L2[0:2])).split("-") + L2[2:]  )
            
            
            list1 = (Table[1].find_all('tr')[3].text[0:5].split("-"))
            list2 = (doc.find_all('span',{'class':'currency__OneLineSpan-sc-4472af-3 bzUWsw'})[0].text.split("-"))
            if(len(list2[0].split(','))>=3):
                list2 = list2[0].split(',')[0:3] + ['...']
                list2 = (','.join(list2)).split('-')
                
            stats_curr_table.add_row(list1+list2)
        
        
        
        else: 
            i = 0
            if(Table[1].find_all('tr')[0].text[0:9] == 'Nicknames'):
                stats_curr_table.add_row((Table[1].find_all('tr')[0].text[0:9] + " " + Table[1].find_all('tr')[0].text[9:]).split(" ",1))
                i+=1;
            
            L0 = (Table[1].find_all('tr')[i].text[0:5] + " " + Table[1].find_all('tr')[i].text[5:]).split(" ",1)
            if(re.search('Rarely',L0[1:][0])):
                x = L0[1:][0].find('Rarely')
                stats_curr_table.add_row(" ".join(L0[0:1]).split("-") + (L0[1:][0][:x] + " " + L0[1:][0][x:]).split('-'))
            else:
                stats_curr_table.add_row((Table[1].find_all('tr')[i].text[0:5] + " " + Table[1].find_all('tr')[i].text[5:]).split(" ",1))


            i+=1

            L1 = (Table[1].find_all('tr')[i].text[0:10] + " " + Table[1].find_all('tr')[i].text[10:]).split(" ",2)

            if(re.search('Rarely',L1[2:][0])):
                x = L1[2:][0].find('Rarely')
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + (L1[2:][0][:x] + " " + L1[2:][0][x:]).split("-"))
            else:
                stats_curr_table.add_row((" ".join(L1[0:2])).split("-") + L1[2:]  ) 
            i+=1

            L2 = (Table[1].find_all('tr')[i].text[0:12] +" " + Table[1].find_all('tr')[i].text[12:]).split(" ",2)
            stats_curr_table.add_row((" ".join(L2[0:2])).split("-") + L2[2:]  )
            i+=1
            list1 = (Table[1].find_all('tr')[i].text[0:5].split("-"))
            list2 = (doc.find_all('span',{'class':'currency__OneLineSpan-sc-4472af-3 bzUWsw'})[0].text.split("-"))
            if(len(list2[0].split(','))>3):
                list2 = list2[0].split(',')[0:3] + ['...']
                list2 = (','.join(list2)).split('-')
                
            stats_curr_table.add_row(list1+list2)
            
        return(stats_curr_table)
    
    
 

In [866]:
 def stats_curr(c1,c2,df):
        
        c1_link = df[df['Code'] == c1]['Links'].values[0]
        
        c2_link = df[df['Code'] == c2]['Links'].values[0]
        
        response = requests.get(c1_link)
        doc1 = BeautifulSoup(response.text, 'html.parser')
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(c1_link))
        
        response = requests.get(c2_link)    
        doc2 = BeautifulSoup(response.text, 'html.parser')
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(c2_link))
        
        if(c1 == 'XBT'):    
            print(stat1(doc1))
            print(stat1(doc2))
            print(stat2(doc2))
        elif(c2 == 'XBT'):
            print(stat1(doc1))
            print(stat1(doc2))
            print(stat2(doc1))
        else:   
            print(stat1(doc1))
            print(stat1(doc2))
            print(stat2(doc1))
            print(stat2(doc2))
        return
        

In [874]:
def renkon():
    url = 'https://www.xe.com/currency/'
    response = requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    cur_df = currency_info(doc)
    print('\t\t\t\tRENKON: CURRENCY CONVERTER\n')
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(tabulate(cur_df[['Name','Code']], headers='keys', tablefmt='psql',showindex = False))
    y = '1'    
    while(y != '0'):    
        user_code1 = input('From: Enter Currency Code (Refer table above for functional codes)')    
        user_code2 = input('To: Enter Currency Code (Refer table above for functional codes)')
        status = 1
        if cur_df[cur_df['Code'] == user_code1.upper()].empty:
            print('Please enter correct code, {} doesn\'t exist.'.format(user_code1.upper()))
            status = 0
        elif cur_df[cur_df['Code'] == user_code2.upper()].empty:
            print('Please enter correct code! {} doesn\'t exist.'.format(user_code2.upper()))
            status = 0
        if(status == 1):
            user_m1 = input('Enter the amount you have\n')
            money = conversion(user_code1,user_code2,user_m1)
            print('It\'s {}\n'.format(money))
            stats_conv(user_code1,user_code2,user_m1)
            stats_curr(user_code1,user_code2,cur_df)
        
        
        print('Do you want to try once again?\n')
        y = input('Enter 0 to stop OR any other number to continue.\n')
    print('Thank you for using RENKON!')    
    return

In [877]:
renkon()

				RENKON: CURRENCY CONVERTER

+--------------------------------+--------+
| Name                           | Code   |
|--------------------------------+--------|
| Emirati Dirham                 | AED    |
| Afghan Afghani                 | AFN    |
| Albanian Lek                   | ALL    |
| Armenian Dram                  | AMD    |
| Dutch Guilder                  | ANG    |
| Angolan Kwanza                 | AOA    |
| Argentine Peso                 | ARS    |
| Australian Dollar              | AUD    |
| Aruban or Dutch Guilder        | AWG    |
| Azerbaijan Manat               | AZN    |
| Bosnian Convertible Mark       | BAM    |
| Barbadian or Bajan Dollar      | BBD    |
| Bangladeshi Taka               | BDT    |
| Bulgarian Lev                  | BGN    |
| Bahraini Dinar                 | BHD    |
| Burundian Franc                | BIF    |
| Bermudian Dollar               | BMD    |
| Bruneian Dollar                | BND    |
| Bolivian Bolíviano             | BOB    |


+-----------------------------------+
|        British Pound Stats        |
+-------------------+---------------+
|                   |     Details   |
+-------------------+---------------+
|        Name       | British Pound |
|       Symbol      |       £       |
|     Minor unit    | 1/100 = penny |
| Minor unit symbol |       p       |
+-------------------+---------------+
+----------------------------------+
|        Yemeni Rial Stats         |
+-------------------+--------------+
|                   |    Details   |
+-------------------+--------------+
|        Name       | Yemeni Rial  |
|       Symbol      |      ﷼       |
|     Minor unit    | 1/100 = Fils |
| Minor unit symbol |     Fils     |
+-------------------+--------------+
+-------------------------------------------------------------------+
|                       British Pound Profile                       |
+--------------+----------------------------------------------------+
|              |                       D